# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("../output_files/city.csv")
file.dropna()
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,New Norfolk,55,AU,1569425354,66,-42.78,147.06,50.00,11.41
1,1,Isiro,100,CD,1569425356,78,2.77,27.62,78.93,3.24
2,2,Torbay,90,CA,1569425355,100,47.66,-52.73,50.00,17.22
3,3,Susanville,1,US,1569425356,76,40.42,-120.65,57.99,3.36
4,4,Walvis Bay,0,NaN,1569425356,68,-22.95,14.51,66.20,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,Rikitea,0,PF,1569425361,67,-23.12,-134.97,71.01,4.90
29,29,Tres Arroyos,0,AR,1569425363,40,-38.38,-60.28,71.55,6.35
37,37,Hermanus,0,ZA,1569425366,44,-34.42,19.24,77.00,10.00
98,98,Miyako,0,JP,1569425382,78,33.63,130.94,71.60,8.05
108,108,Paso de Carrasco,0,UY,1569425385,31,-34.86,-56.05,77.00,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -23.1206197,
                    "lng": -134.9687529
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.11945897010728,
                        "lng": -134.9673878201073
                    },
                    "southwest": {
                        "lat": -23.12215862989272,
                        "lng": -134.9700874798928
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "60864d02c84a997a4e41d2ee16ce4b24b3e67a6a",
            "name": "Pension Maro'i",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2304,
                    "html_attributions": [
                   

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -38.3693062,
                    "lng": -60.29775069999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -38.36800572010728,
                        "lng": -60.29633492010727
                    },
                    "southwest": {
                        "lat": -38.37070537989273,
                        "lng": -60.29903457989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "0f366c6d0df67a14e987d543c3ce1a081fe4b783",
            "name": "Elegance Hotel Tres Arroyos",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 3096,
                    "html_attributions": [


{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAKMMiBqMBzydIzca-xM4-XnuHDtQBN_2SSPLi64Q44TYziCGeLSFJLKsF6QkSsJOnbX7LJodArLZWAQOXBJakAOqkMzZU1kUNPjHeKF0pWNlalpAL21YQPDeldqw1c24KG6fAj3bauq4e4TiRECOXk86qhaA2s4yGz-LNUStbZp2iwcqUGoNNQI8Vy_QexAoBabj1HsraF0eUVe0aj3NQYHNQntTnc3iCrbuPdCFjPdQYKxvf3Pky_reXGarQMu1T4fekP8nFqlw-RXOKFTKmkWLkUrVUYXzs0_LJskvip6XlNZ-hCgpm9l0_TYkKpcAgIq127X9R2OH0dQqg38gEcxIWo0RMCw5NBjl2dGU-kOe3JIZ-9jBY8Na5HNLwYjotggA-QkzSsj89xfapcmB1bISEAxWMrIJefGL66yj5-qEdVAaFAJdLZprtpqvqPXa0BJYS2k6M2nE",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.4169,
                    "lng": 19.17732
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.41555082010727,
                        "lng": 19.17867752989272
                    },
                    "southwest": {
                        "lat": -34.41825047989272,
                        "

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAC4IA0h0BqENSNNC7HhIM2S4XinfP4j1okz_v6nMYCYQ3XETJI7qzTZQRbVgALuebutOnLeviTu5DIKXf4JxuUwdT4WNVskQDxI55rC-UD49o6uLiHFKtvns1usVhqBHJ7eA8Edr_EEj5QOG4Sr6Dv0-f8mt-Fl-loEnEqTwfcJlBknRQ3nqnH75QI0iZVb5yIKtUHIcQbv3weaB7l5ItUk1a8kpAZeSmT3rT5oBo6mjpSynPhX9oyLulSZT7R7aA6QBevNM58KfjMYaQ6m5AATeq42OHSzv7i7GP-LuFzEy6DjBs_dDt_aY3WS3dfNV1Mb04YIX_Fu61c5GMeWJsf4mTBsv--41284Rj_zxHHlsS92AgI0PRzkVP8ferxjSieoG5S-F2QeasQy9p3jnAB4SECqI93VFL-WFNy0d49TCFMIaFEgar5a6EqjnX-CjAwZhMmVt5W9V",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 33.5909549,
                    "lng": 130.412541
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.59230472989272,
                        "lng": 130.4138908298928
                    },
                    "southwest": {
                        "lat": 33.58960507010728,
                       

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAPCZjunbEJdLoOVWDFX3DbWsnBuWOqTT9U80wLuXkD4otIFQB0UQYQKqRqd8oumysPnLcW9FlZ6CuzmMhP-bnreOs4Nni_vq198p9weSqpl-24DthpUsFL9lwYIcvTBnbip_2zTJ8yoI1v_p0cBxItHN_pGQtp6lB73NU56VHi4SA69qFRH7CCf8PpJmAN-eWY4CqZIRdzbNDwQqGOxa6AoGApMo41GhdRvjawocGkKjV0K-XYcpne6nK8VxA53Qsd8GnlBdDmYn5dgdm4RcBx2O9GhOSNFJgEEtwJk76cyuMeXzTvE8qvNQyzHTYHIGbETpOi94DW7MaQYs8G6K8eytoFnZfn8UTO0yDsxVVoeNW1rXfK-NLEz1GNpYjkmuXtNd5NxfJsCBdOwGie9BZF8SEM5-TuK9h6SERfTgl_UrA-YaFFBFgvW0_dL5_v0o71miD3NUq5jT",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.890696,
                    "lng": -56.055312
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.88938212010728,
                        "lng": -56.05394262010728
                    },
                    "southwest": {
                        "lat": -34.89208177989272,
                    

...

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -19.5437765,
                    "lng": 45.4498755
                },
                "viewport": {
                    "northeast": {
                        "lat": -19.54222677010728,
                        "lng": 45.45099767989272
                    },
                    "southwest": {
                        "lat": -19.54492642989272,
                        "lng": 45.44829802010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "bb4a8ddec70a274028331f77c84088ec9a268628",
            "name": "Princesse Tsiribihina",
            "photos": [
                {
                    "height": 1360,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/101602497891877256043/p

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAL9arFN77rwdZMHPJxQHncL7SLTU1Bu7m4dubeKhozWQYqZq4LnYpU38d8TNKgb3uIe7hA_61Trm4yQ_Tn4hNzOdskEB4-gOnOrcoYaRJ6wr5ZKf9aRgvloSYf5_pI53IcnzPgCX0H3c-P3Fd2ophQW9gJC1Yt8T0-iDAk1qdQpQkoHjKo1dESHonimle6OThZKMNOuRk-yce9zr73oGeyL5QPht7KRGtJl-KAqB2toTa11i-sZnUp1wduXeU8TCQ82AyotTdnP_1Rg4DKewkqsSxFVVRMWHF8MH1T2qfvoEqu54DngeGerlsYsQRkMi9vS94eRX6EhznCCM1ojivMOEVYmY_xHYpuo4PZ09Ytga7Cp3x3GW_VG6SC7VKyO-Bee5KTtQfhOqRAEwjXJeZZISEPKJaa1qvHGNCWR0DA-iNmsaFHFUA7VgDMVZEesxSobOS9tJ4kav",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -8.730362999999999,
                    "lng": 121.824188
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.729604149999997,
                        "lng": 121.8258692298927
                    },
                    "southwest": {
                        "lat": -8.732639549999998,
             

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAGCcTx1_l9vCpG9_YzUDoySBip3EMB43DwZAmudnKUzJ56bAxWQLWyOhq91UhJtSFqZdZ_ZFKeJzALU6n_PFBWmwDtEBiII2bYwRxNMwTVgSbmdmFjpUlduJnO_WujFo0ySwGjnEJDLy2T5O4rgKeLQQmTC7EucG18H047FzgLex20BHCflxPhzG0f1_MnSJlu2Of9GwLOT84xRcYBNbF3zPUWZgolU9ZfvJUsOCz0KIbHhjv0S6qYaNBBs9f8_YZ0Cdfbn4bD7md0MJwtKNRMmD3PKy0G675QWUU7iYRbOLuD3j5BjoDsfckFVjuUCSbuNLO5Gsj8bb8XV8Aybk3p_LPv5yXSQiKMqw9IAUGHeFQCS2pNaPBofFyQsfDfrML3oChxfcOLnOWQV7n0CNmEsSEMp5iY6OLpgtGtFJ7CyoVWkaFGhAkHJspjUdvGhWVsP5gWtNTNaF",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 39.872574,
                    "lng": 45.35704399999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 39.87381382989273,
                        "lng": 45.35838277989271
                    },
                    "southwest": {
                        "lat": 39.87111417010728,
                 

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAHLQ_8i9RzVrb8Md7spFQ6RDdBIOWnItF8FiNIv8io9EF-LHPO-ymCpszZfZMPEz43EhvdQ1PJwdEtfN4QeuaT3io3ZIALrfiaPdWKSqItmwRIKW8HY78JoU4hKG8zWDqjduBOrD4rugMsOGkv4sVu_Fr1QoiR9yBMmWkj-mYykmC18qOg73QAWzwEgbPFQr6kwk9foh6e5gQ05ib5R0BnDBXa5hkoDdEpoBKPzG0CB7w2h3pckT-p2U_vJpeKNpead9ItOFWwKNg0vmZPuSWJRT3aN4FNRB6Ck-5kXtcxIZKexImi5Th7t65mJ9zBVhdZqArqzvzi1u9lxia7WMFgwjbMXWTmmXDUUNr2UDkyNin_XfoCatGxbOrUtzfAcXjJHWawh7JF8TOq3UGPfbaBISENQDSYgVFoeEey0_CXhlaKoaFKC-51ayooZwfXawqyiPEjLwKJes",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 42.8168371,
                    "lng": 93.520449
                },
                "viewport": {
                    "northeast": {
                        "lat": 42.81818932989272,
                        "lng": 93.52197687989273
                    },
                    "southwest": {
                        "lat": 42.81548967010728,
                        

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,20,Rikitea,0,PF,1569425361,67,-23.12,-134.97,71.01,4.90,Pension Maro'i
1,29,Tres Arroyos,0,AR,1569425363,40,-38.38,-60.28,71.55,6.35,Elegance Hotel Tres Arroyos
2,37,Hermanus,0,ZA,1569425366,44,-34.42,19.24,77.00,10.00,Kennedys Beach Villa
3,98,Miyako,0,JP,1569425382,78,33.63,130.94,71.60,8.05,FP HOTELS Fukuoka-Hakata
4,108,Paso de Carrasco,0,UY,1569425385,31,-34.86,-56.05,77.00,9.17,Hotel Sofitel Montevideo Casino Carrasco and Spa
5,117,Namibe,0,AO,1569425388,88,-15.19,12.15,70.47,9.48,Hotel ibis Styles Iu Namibe
6,118,Barrow,0,AR,1569425389,40,-38.31,-60.23,71.55,6.35,Hotel El Cortijo
7,136,Sayat,0,TM,1569425393,33,38.78,63.88,77.00,2.24,Hotel Yupek Yoly (Silk Way)
8,183,Noumea,0,NC,1569425125,82,-22.28,166.46,73.99,2.24,Nouvata
9,210,Tiznit,0,MA,1569425325,81,29.70,-9.73,70.47,4.34,Dar Najmat


In [8]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))